In [ ]:
df = read.csv('csv/timeseries.csv')

In [ ]:
str(df)

In [ ]:
# 순번
df[['X']] <- 1:nrow(df)

In [ ]:
# 선형회귀
reg <- lm(매출액~X,data=df)
reg

In [ ]:
# 추세변동
T = df[['X']]*reg$coefficients[2] + reg$coefficients[1]

In [ ]:
# 매출액과 추세변동의 그래프
plot(df[['X']],df[['매출액']],type='p')
lines(df[['X']],df[['매출액']])
lines(df[['X']],T)

In [ ]:
library(dplyr)
library(zoo)

In [ ]:
df[['사분기이동평균']] <- rollmean(df[['매출액']],4, fill=NA, align='center')

In [ ]:
df[['중심이동평균']] <- rollmean(df[['사분기이동평균']],2, fill=NA, align='right')

In [ ]:
df[['특정계절지수']] <- df[['매출액']] / df[['중심이동평균']]

In [ ]:
# 계절별 평균
group <- group_by(df,분기)
s_coeff <- summarise(group,대표계절지수=mean(특정계절지수,na.rm=T))
s_coeff

In [ ]:
df[['대표계절지수']] <- c(s_coeff[['대표계절지수']], rep(NA,nrow(df)-nrow(s_coeff)))


In [ ]:
df[['수정된대표계절지수']] <- df[['대표계절지수']] / sum(df[['대표계절지수']],na.rm=T) * 4

In [ ]:
df[['계절변동']] <- rep(df[['수정된대표계절지수']][1:4],5)

In [ ]:
df

In [ ]:
S <- df[['계절변동']]

In [ ]:
# 계절변동의 그래프
plot(df[['X']],S,type='p')
lines(df[['X']],S)

In [ ]:
# 순환 및 불규칙 변동
CI <- df[['매출액']] / T / S

In [ ]:
plot(df[['X']],CI,type='p')
lines(df[['X']],CI)

In [ ]:
# 분석결과 수집
df2 <- df['매출액']
df2[['추세변동']] <- T
df2[['계절변동']] <- S
df2[['순환및불규칙변동']] <- CI

In [ ]:
df2[['순환변동']] <- rollmean(df2[['순환및불규칙변동']],3, fill=NA, align='right')

In [ ]:
df2[['불규칙변동']]  <- CI / df2[['순환변동']] 

In [ ]:
df2

In [ ]:
# 순환변동의 그래프
plot(df[['X']],df2[['순환변동']],type='p')
lines(df[['X']],df2[['순환변동']])

In [ ]:
# 불규칙변동의 그래프
plot(df[['X']],df2[['불규칙변동']],type='p')
lines(df[['X']],df2[['불규칙변동']])